In [1]:
import os

import tensorflow as tf
from tensorflow import keras

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0  # -1 表示 unspecified value，即该维度不用指定，系统会自动计算。
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [3]:
# 定义一个简单的序列模型
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# 创建一个基本的模型实例
model = create_model()

# 显示模型的结构
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [4]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 创建callback来保存模型的权重
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# 在模型训练时使用回调
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 832/1000 [=======================>......] - ETA: 0s - loss: 1.2322 - accuracy: 0.6683
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 1ms/sample - loss: 1.1528 - accuracy: 0.6870 - val_loss: 0.7667 - val_accuracy: 0.7710
Epoch 2/10
 800/1000 [=======================>......] - ETA: 0s - loss: 0.4390 - accuracy: 0.8750
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 329us/sample - loss: 0.4245 - accuracy: 0.8790 - val_loss: 0.5552 - val_accuracy: 0.8330
Epoch 3/10
 800/1000 [=======================>......] - ETA: 0s - loss: 0.2782 - accuracy: 0.9250
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 308us/sample - loss: 0.2754 - accuracy: 0.9280 - val_loss: 0.4898 - val_accuracy: 0.8480
Epoch 4/10
 800/1000 [=======================>......] - ETA: 0s - loss: 0.1988 - accuracy: 0.9488
Epoch 0

In [5]:
# 创建模型实例
model = create_model()

# 评估模型
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 2.2833 - accuracy: 0.1310
Untrained model, accuracy: 13.10%


In [6]:
# 加载权重
model.load_weights(checkpoint_path)

# 重新评估模型
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.4213 - accuracy: 0.8670
Restored model, accuracy: 86.70%


In [17]:
# 使用 format（）方法实现动态命名 
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 定义一个回调函数实现每5个epoch保存一次检查点文件
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    monitor='val_loss',
    verbose=1, 
    save_weights_only=True,
    mode='auto',
    period=5) # 根据警告，换为save_freq，是逐个保存，可能存在bug；

# 创建一个新的模型实例
model = create_model()

# 传入 checkpoint_path 参数，实现将检查点文件保存到该路径
model.save_weights(checkpoint_path.format(epoch=0))

# 使用自定义的回调函数训练模型
model.fit(train_images, 
          train_labels,
          epochs=50, 
          callbacks=[cp_callback],
          validation_data=(test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [18]:
# 加载最后一个检查点文件
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2\\cp-0050.ckpt'

In [19]:
# 创建一个新的模型实例
model = create_model()

# 加载之前保存的权重（检查点文件）
model.load_weights(latest)

# 重新评估模型
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.4769 - accuracy: 0.8760
Restored model, accuracy: 87.60%


In [20]:
# 保存权重
model.save_weights('./checkpoints/my_checkpoint')

# 创建一个新的模型实例
model = create_model()

# 恢复模型
model.load_weights('./checkpoints/my_checkpoint')

# 评估模型
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 - 0s - loss: 0.4769 - accuracy: 0.8760
Restored model, accuracy: 87.60%


In [50]:
# 创建模型实例并训练
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# 使用 save 方法保存模型
path='saved_model'
isExists = os.path.exists(path)
if not isExists:
    # 如果不存在则创建目录
    os.makedirs(path)
model.save(path) 

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 1s 544us/sample - loss: 1.1565 - accuracy: 0.6800
Epoch 2/5
1000/1000 [==============================] - 0s 141us/sample - loss: 0.4090 - accuracy: 0.8810
Epoch 3/5
1000/1000 [==============================] - 0s 145us/sample - loss: 0.2742 - accuracy: 0.9310
Epoch 4/5
1000/1000 [==============================] - 0s 144us/sample - loss: 0.2238 - accuracy: 0.9400
Epoch 5/5
1000/1000 [==============================] - 0s 155us/sample - loss: 0.1561 - accuracy: 0.9640
INFO:tensorflow:Assets written to: saved_model\assets


In [51]:
items = os.listdir(os.chdir(path))
print(os.listdir('.'))

['assets', 'saved_model.pb', 'variables']


In [54]:
# 切换回根目录
os.chdir('..')
# 加载保存的模型
new_model = tf.keras.models.load_model(path)
# 检查保存模型的网络架构
new_model.summary()

Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_58 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_29 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_59 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [55]:
# 评估恢复的模型
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

1000/1000 - 0s - loss: 0.4072 - accuracy: 0.8690
Restored model, accuracy: 86.90%
(1000, 10)


In [56]:
# 创建新的模型实例并训练
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# 将整个模型（包括网络结构和权重）保存为 HDF5 文件
# 使用save方法通过设置文件后缀名为 '.h5' 来实现保存为 HDF5 文件
path='h5_format_model'
isExists = os.path.exists(path)
if not isExists:
    os.makedirs(path)
    
model_name = f'{path}/my_model.h5'
model.save(model_name) 

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 1s 532us/sample - loss: 1.1811 - accuracy: 0.6570
Epoch 2/5
1000/1000 [==============================] - 0s 141us/sample - loss: 0.4246 - accuracy: 0.8750
Epoch 3/5
1000/1000 [==============================] - 0s 140us/sample - loss: 0.2864 - accuracy: 0.9280
Epoch 4/5
1000/1000 [==============================] - 0s 143us/sample - loss: 0.2125 - accuracy: 0.9500
Epoch 5/5
1000/1000 [==============================] - 0s 136us/sample - loss: 0.1627 - accuracy: 0.9640


In [57]:
# 加载保存的整个模型
new_model = tf.keras.models.load_model(model_name)

# 打印网络架构摘要
new_model.summary()

Model: "sequential_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_60 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_30 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_61 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [58]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

1000/1000 - 0s - loss: 0.4223 - accuracy: 0.8640
Restored model, accuracy: 86.40%
